In [5]:
import pandas as pd
import re
from typing import Dict

In [6]:
# Cargar el CSV (actualiza 'ruta_del_csv.csv' por la ruta real de tu archivo)
df_train = pd.read_csv(r"..\data\interim\cvs_texto_train.csv", sep=',', encoding='utf-8')
df_test = pd.read_csv(r"..\data\interim\cvs_texto_test.csv", sep=',', encoding='utf-8')

def limpiar_texto(texto):
    # Asegurarse de que el valor es una cadena
    texto = str(texto)
    # Eliminar saltos de línea innecesarios y reemplazarlos por un espacio
    texto = re.sub(r'\s*\n\s*', ' ', texto)
    # Reducir múltiples espacios a uno solo
    texto = re.sub(r'\s{2,}', ' ', texto)
    # Eliminar espacios al inicio y al final
    texto = texto.strip()
    return texto

In [7]:
# Aplicar la función de limpieza a la columna de texto
df_train['texto_normalizado'] = df_train['Texto extraído'].apply(limpiar_texto)
df_test['texto_normalizado'] = df_test['Texto extraído'].apply(limpiar_texto)

print("Proceso de extracción y normalización completado.")

Proceso de extracción y normalización completado.


In [8]:
# Mostrar algunas observaciones para comparar el texto original y el normalizado
pd.set_option('display.max_colwidth', None)  # Mostrar el texto completo en cada celda
muestras_train = df_train[['Texto extraído', 'texto_normalizado']].sample(5, random_state=1)
muestras_test = df_test[['Texto extraído', 'texto_normalizado']].sample(5, random_state=1)

print(muestras_train)
print(muestras_test)

In [9]:
def segmentar_texto(texto: str) -> Dict[str, str]:
    """
    Segmenta un texto de CV en secciones clave: experiencia, formación, habilidades, otros.
    """
    secciones: Dict[str, str] = {
        'experiencia': '',
        'formacion': '',
        'habilidades': '',
        'otros': ''
    }

    patrones: Dict[str, str] = {
        'experiencia': r'(Experiencia laboral|Experiencia)',
        'formacion': r'(Formación académica|Educación|Formación)',
        'habilidades': r'(Habilidades|Competencias)'
    }

    matches = []
    for key, pattern in patrones.items():
        for match in re.finditer(pattern, texto, re.IGNORECASE):
            matches.append((match.start(), key))
    matches.sort(key=lambda x: x[0])

    if matches:
        primer_inicio = 0
        primer_pos, _ = matches[0]
        if primer_pos > primer_inicio:
            secciones['otros'] = texto[primer_inicio:primer_pos].strip()

        for i, (inicio, seccion) in enumerate(matches):
            fin = matches[i+1][0] if i+1 < len(matches) else len(texto)
            contenido = texto[inicio:fin].strip()
            secciones[seccion] = contenido
    else:
        secciones['otros'] = texto.strip()

    return secciones


In [10]:
# Aplicar la segmentación a cada fila del DataFrame
df_train_segmentado = df_train['texto_normalizado'].apply(segmentar_texto).apply(pd.Series)
df_test_segmentado = df_test['texto_normalizado'].apply(segmentar_texto).apply(pd.Series)

# Puedes unirlo al DataFrame original si quieres
df_train = pd.concat([df_train, df_train_segmentado], axis=1)
df_test = pd.concat([df_test, df_test_segmentado], axis=1)

In [11]:
# Mostrar una muestra de segmentos detectados
df_train[['Nombre del archivo', 'experiencia', 'formacion', 'habilidades', 'otros']].sample(3, random_state=1)

,Nombre del archivo,experiencia,formacion,habilidades,otros
19,CV_02_cdatos_02.pdf,"Experiencia Profesional Director de Data & Analytics | QuantumTech Advisors | 2020 - Actualidad - Dirección estratégica de la unidad de Data & AI con más de 80 personas. - Diseño de plataformas de datos para clientes del sector financiero y retail. - Creación de partnerships tecnológicos con Oracle, IBM y Snowflake. - Incremento del volumen de negocio en un 40% en 3 años. Consultor Senior de Trans",Formación Académica Executive MBA | IE Business School | 2018 Grado en Estadística e Ingeniería de Datos | Universidad Carlos III de Madrid | 2010,"Habilidades - Dirección estratégica de Data & AI - Analítica predictiva y machine learning - Arquitectura de plataformas cloud - Desarrollo comercial y relaciones cliente - Gestión de equipos multidisciplinares - Inglés avanzado (C1), Francés intermedio (B2)","Curriculum Vitae Nombre: Marcos Vidal Ortega Email: marcos.vidal@example.com Teléfono: +34 612 987 321 Ubicación: Madrid, España LinkedIn: linkedin.com/in/marcosvidalai Perfil Profesional Profesional senior con más de 14 años de"
55,CV_04_cdatos_02.pdf,Experiencia Profesional Responsable de Inteligencia de Datos | NexaTech Group | 2020 - Actualidad - Dirección de soluciones de IA y analítica avanzada para banca y seguros. - Supervisión de arquitecturas de datos escalables en AWS y Azure. - Implementación de soluciones de NLP y análisis predictivo. - Coordinación de un equipo de 50 personas y liderazgo del roadmap estratégico. Consultor en Data & Analytics | DataPulse Iberia | 2013 - 2020 - Desarrollo de modelos de machine learning y segmentación de clientes. - Diseño de pipelines de datos y soluciones de BI. - Participación en proyectos internacionales en colaboración con equipos de LATAM.,Formación Académica Máster en Inteligencia Artificial | Universidad de Sevilla | 2016 Grado en Ingeniería de Telecomunicaciones | Universidad de Zaragoza | 2012,"Habilidades - Inteligencia Artificial aplicada a negocio - Machine Learning y NLP - Arquitectura de datos en la nube - Liderazgo y gestión de roadmap tecnológico - Gobierno del dato y calidad - Inglés profesional (C1), Italiano intermedio (B1)","Curriculum Vitae Nombre: Ignacio Ríos Delgado Email: ignacio.rios@example.com Teléfono: +34 633 852 479 Ubicación: Madrid, España LinkedIn: linkedin.com/in/ignacioriosdata Perfil Profesional Profesional orientado a la innovación con sólida"
10,CV_01_jurista_02.pdf,Experiencia Profesional Abogada Senior en Derecho Urbanístico Bufete Legal & Urbanismo | Málaga | Febrero 2020 - Actualidad - Elaboración de due diligence urbanísticas para suelos y edificaciones. - Tramitación de autorizaciones administrativas y medioambientales. - Redacción de proyectos de reparcelación y expedientes de expropiación. - Participación en secretarías técnicas de Juntas de Compensación. - Preparación de recursos administrativos y contencioso-administrativos. Abogada Junior en Derecho Administrativo Urbanlex Abogados | Sevilla | Enero 2017 - Enero 2020 - Asesoramiento legal en contratos administrativos y licitaciones públicas. - Apoyo en seguimiento legislativo y elaboración de informes técnicos. - Gestión de expedientes urbanísticos y planeamiento territorial.,Formación Académica Máster en Derecho Urbanístico y del Medio Ambiente Universidad de Málaga | 2016 - 2017 Grado en Derecho Universidad de Sevilla | 2011 - 2015 Idiomas Español: Nativo Inglés: Avanzado (C1),Habilidades - Derecho Urbanístico - Contratación administrativa - Tramitación de licencias y autorizaciones - Redacción de informes jurídicos - Trabajo en equipo - Microsoft Office y herramientas legales digitales,"Currículum Vitae Datos Personales Nombre: Laura Martínez Rodríguez Dirección: Calle Limonar, 27, 29016 Málaga Teléfono: +34 654 321 987 Correo electrónico: laura.martinez@example.com Fecha de nacimiento: 12 de marzo de 1990 Nacionalidad: Española Perfil Profesional Abogada especializada en Derecho Público y Urbanismo co

In [12]:
# Mostrar una muestra de segmentos detectados
df_test[['Nombre del archivo', 'experiencia', 'formacion', 'habilidades', 'otros']].sample(3, random_state=1)

,Nombre del archivo,experiencia,formacion,habilidades,otros
3,cv03.pdf,"Experiencia laboral Miembro del departamento E-Powertrain de eUsRacing Escudería Motociclismo (Universidad de Sevilla) | Enero 2024 - Actual Sobre mí Diseño 3D y desarrollo de una batería eléctrica de alto rendimiento para motocicleta de competición. Me defino como una persona con Participación en la fabricación y validación del sistema para la competición ganas de aprender y proactividad, internacional Motostudent, celebrada en el circuito de MotoGP de Aragón. por ello decidí estudiar ingeniería. Siempre intento ayudar a quien lo Ayudante en Ventas necesite de ahí que ejerza como HERMANOS SALGUERO MARÍN, S.L. | Enero 2021 - Septiembre 2024 voluntario en diferentes Participación en el aumento de rentabilidad en los procesos técnicos. proyectos. Aprendizaje de nociones básicas en la reparación de vehículos Voluntario 3,2,1 Sputnik (Fundación El Mundo Que Viene) Contacto Voluntario de los programas Sputnik | Septiembre 2022- Actualidad Planificación de todas las actividades desarrolladas durante el +34 626 225 159 programa. Coordinación de todo el grupo de voluntarios para poder conseguir el samuel.salguero.munoz@gmail.com éxito del evento. Alcalá de Guadaíra, Sevilla.","formación Programa Erasmus + realizado en Kusadasi,Turquía | Marzo 2023 Seleccionado como uno de los seis representantes de España para - Carné de conducir. debatir sobre el futuro de los jóvenes. - Disponibilidad finde semanas Programa Curious Minds (Fundación Valentín de Madariaga) Curso en Lean StartUp, Liderazgo y Pensamiento Crítico | Octubre 2022 - Diciembre 2022",Habilidades Idiomas Diseño 3D (SolidWorks) Castellano : Nativo. Programas de Simulación: Fotovoltaica: PVsyst Inglés: Nivel B2 (Preparación examen) Termosolar: SAM,Samuel Salguero Estudiante de Ingeniería de la Energía
7,cv07.pdf,experiencia práctica y aplicar mis,"EDUCACIÓN 609 269 744 Grado en Economía, Mención en Análisis iremorlis@gmail.com Económico Cuantitativo Universidad de Sevilla, WWW.LINKEDIN.COM/IN/IRENE- 2020-2024 MORALES-LISSÉN-9374B22A9 Nota media del expediente: 7,07 ACERCA DE MÍ","habilidades en un entorno desafiante. prácticas: Análisis económico y de mercado. APTITUDES Estrategia y planificación económica. Gestión y composición de programas. Eficiencia en la gestión del tiempo y Gestión de relaciones con clientes y recursos proveedores. Trabajo en equipo Soporte en proyectos de desarrollo Capacidad para trabajar bajo presión empresarial. Organización y planificación Capacidad de aprendizaje continuo Apoyo en tareas contables y financieras. Adaptabilidad y capacidad para Investigación y recopilación de datos. gestionar múltiples tareas simultáneamente IDIOMAS Dominio de herramientas ofimáticas (Word, Power Point y Excel) y software especializado (Gretl) Español (nativo). Inglés, nivel B2.",Irene Morales Lissén GRADUADA EN ECONOMÍA CONTACTO
6,cv06.pdf,"Experiencia Profesional - Árbitro de Baloncesto de la Federación Andaluza de Baloncesto. (2019 - sigo activo) - Voluntariado en la Maratón de Málaga ( en 2021) - Socorrista en urbanización en marbella, en Reserva de Marbella (Período estival) Grupo Mansof - Socorrista en urbanización en Fuengirola, correturnos por varias urbanizaciones (Periodo estival) - Socorrista en Parque acuático Aqualand (Torremolinos) - Clases privadas de",Educación Primaria de inglés durante el curso 2024 -2025. 4. Idiomas - Inglés (B1). - Francés nivel básico. 5. Otros - 11 años federado en natación. - 7 años federados en Baloncesto actualmente.,,Curriculum Vitae 1. In


In [13]:
import os
import re
import pandas as pd
from typing import List, Dict

In [14]:
def extraer_descripcion(offer_text: str) -> str:
    """
    Extrae la descripción de la oferta a partir del texto.
    Busca la sección que sigue a "Descripción:" y, si existe, corta antes de "Descripción del puesto:".
    """
    if "Descripción:" in offer_text:
        desc = offer_text.split("Descripción:", 1)[1]
        # Si se indica una subsección "Descripción del puesto:" se toma solo lo anterior
        if "Descripción del puesto:" in desc:
            desc = desc.split("Descripción del puesto:", 1)[0]
        return limpiar_texto(desc)
    return ""


In [15]:
def procesar_ofertas_archivo(ruta_archivo: str) -> pd.DataFrame:
    """
    Procesa un archivo .txt con varias ofertas separadas por líneas de guiones.
    Retorna un DataFrame con dos columnas:
      - 'nombre_archivo': nombre del archivo de origen.
      - 'descripcion_oferta': la descripción extraída de cada oferta.
    """
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        contenido = file.read()
    
    # Dividir el contenido en ofertas usando como separador líneas con 5 o más guiones
    offers: List[str] = re.split(r'-{5,}', contenido)
    # Eliminar posibles entradas vacías y espacios extra
    offers = [offer.strip() for offer in offers if offer.strip()]
    
    data: List[Dict[str, str]] = []
    for offer in offers:
        descripcion = extraer_descripcion(offer)
        data.append({
            'nombre_archivo': os.path.basename(ruta_archivo),
            'descripcion_oferta': descripcion
        })
    
    return pd.DataFrame(data)



In [16]:
# Ejemplo de uso con un archivo concreto:
ruta_archivo = r"..\data\raw\ofertas\Ciencia_de_datos_España.txt"  # Cambia esta ruta según la ubicación de tu archivo
df_ofertas = procesar_ofertas_archivo(ruta_archivo)

# Mostrar las primeras filas del DataFrame resultante
df_ofertas.head()

,nombre_archivo,descripcion_oferta
0,Ciencia_de_datos_España.txt,"Acerca del empleo Los Ingenieros de Software Junior son profesionales técnicamente capacitados con deseo de diseñar, desarrollar, solucionar problemas y mantener soluciones innovadoras para cuestiones técnicas de producción, incluyendo el desarrollo de scripts y aplicaciones para la manipulación de datos y archivos. El candidato ideal es una persona orientada al trabajo en equipo que prospera en entornos de ritmo rápido, colaborando con otros Ingenieros de Software, Ingenieros de Automatización y Product Managers para desarrollar soluciones creativas en procesos de localización y productos tecnológicos de la empresa. Debe estar automotivado para aprender y tener interés y experiencia en web scraping."
1,Ciencia_de_datos_España.txt,"Acerca del empleo 🚀 Únete a Inetum como Responsable de Data & AI ¿Eres un líder visionario apasionado por la tecnología y la innovación? Esta es tu oportunidad para dirigir nuestra unidad de negocio de Data & AI, transformando industrias y generando impacto real. 🔍 Lo que harás: Liderarás un equipo multidisciplinar, con un alto volumen de negocio. Diseñarás y ejecutarás estrategias innovadoras de Data & AI para sectores clave como banca, telecomunicaciones, retail, salud, ... Potenciarás relaciones con clientes estratégicos y socios tecnológicos líderes del mercado. Gestionarás el P&L de la unidad, impulsando rentabilidad y excelencia. 🌟 Lo que buscamos: Experiencia liderando unidades de negocio en Data & AI o tecnologías afines en consultoría o integración. Experiencia en áreas como Data Governance, Plataformas y Arquitectura, Visualización e IA. Habilidades excepcionales en desarrollo de negocio y gestión de equipos grandes y diversos. Pasión por la tecnología, el impacto social y la transformación digital. Inglés nivel alto (C1 o superior). 🎯 Lo que ofrecemos: Liderar una unidad clave en un entorno dinámico y en constante evolución. Acceso a proyectos estratégicos con clientes líderes y tecnologías de vanguardia. Retribución competitiva y entorno de trabajo flexible y colaborativo. 📍 Ubicación: Madrid, modalidad de trabajo híbrida. Sé parte de la transformación. ¿List@ para liderar el futuro de Data & AI con nosotros? 🌐"
2,Ciencia_de_datos_España.txt,"Acerca del empleo Estamos ampliando nuestro equipo de ciencia de datos para ayudar a Tendam a resolver desafíos complejos en la industria de la moda retail. Actualmente buscamos un/a Data Scientist para una oportunidad única de influir en la forma en que el mundo experimenta la moda. Qué harás: Descomponer problemas complejos de datos y realizar análisis rigurosos y evaluaciones. Llevar a cabo el preprocesamiento de datos estructurados y no estructurados. Extraer y analizar datos de las bases de datos de la empresa para impulsar la optimización y mejora del desarrollo de productos, técnicas de marketing y estrategias comerciales. Desarrollar modelos de datos y algoritmos personalizados para aplicarlos a conjuntos de datos. Presentar la información mediante técnicas de visualización de datos. Qué buscamos: Formación científica con habilidades computacionales y analíticas. Sólidas habilidades para la resolución de problemas. Conocimientos de programación en Python y SQL. Comprensión y experiencia con sistemas de big data, especialmente Spark (PySpark). Conocimiento de diversas técnicas de aprendizaje automático (clustering, regresión, árboles de decisión, redes neuronales, etc.). Experiencia con herramientas y librerías de visualización de datos. Motivación para aprender y dominar nuevas tecnologías y técnicas. Excelentes habilidades de comunicación oral y escrita. ~1 año de experiencia en la industria. Cualificaciones adicionales: Experiencia trabajando con grandes volúmenes de datos. Experiencia con servicios web en la nube (por ejemplo, AWS). Conocimiento y experiencia con sistemas de control de versiones (por ejemplo, Git). Por qué Tendam: Uno de los mayores repositorios de datos de

In [17]:
print(df_ofertas['descripcion_oferta'].sample(5, random_state=1).to_string(index=False))

                                                                                                                   Acerca del empleo Estamos ampliando nuestro equipo de ciencia de datos para ayudar a Tendam a resolver desafíos complejos en la industria de la moda retail. Actualmente buscamos un/a Data Scientist para una oportunidad única de influir en la forma en que el mundo experimenta la moda. Qué harás: Descomponer problemas complejos de datos y realizar análisis rigurosos y evaluaciones. Llevar a cabo el preprocesamiento de datos estructurados y no estructurados. Extraer y analizar datos de las bases de datos de la empresa para impulsar la optimización y mejora del desarrollo de productos, técnicas de marketing y estrategias comerciales. Desarrollar modelos de datos y algoritmos personalizados para aplicarlos a conjuntos de datos. Presentar la información mediante técnicas de visualización de datos. Qué buscamos: Formación científica con habilidades computacionales y analíticas. S

In [19]:
from pathlib import Path

# Ruta a la carpeta de archivos .txt
carpeta_ofertas = Path("../data/raw/ofertas")

# Obtener todos los archivos .txt en la carpeta
archivos_txt = list(carpeta_ofertas.glob("*.txt"))

# Diccionario donde se guardará un DataFrame por archivo
dataframes_por_archivo = {}

# Procesar cada archivo individualmente
for ruta in archivos_txt:
    nombre_archivo = ruta.stem  # nombre sin extensión
    df = procesar_ofertas_archivo(str(ruta))  # aplica tu función
    dataframes_por_archivo[nombre_archivo] = df  # guarda en el diccionario

# Verificar: por ejemplo, mostrar las primeras filas del DataFrame del primer archivo
for nombre, df in dataframes_por_archivo.items():
    print(f"\nArchivo: {nombre}")
    display(df.head())  # Si estás en Jupyter, muestra bonito



Archivo: Ciencia_de_datos_España


,nombre_archivo,descripcion_oferta
0,Ciencia_de_datos_España.txt,"Acerca del empleo Los Ingenieros de Software Junior son profesionales técnicamente capacitados con deseo de diseñar, desarrollar, solucionar problemas y mantener soluciones innovadoras para cuestiones técnicas de producción, incluyendo el desarrollo de scripts y aplicaciones para la manipulación de datos y archivos. El candidato ideal es una persona orientada al trabajo en equipo que prospera en entornos de ritmo rápido, colaborando con otros Ingenieros de Software, Ingenieros de Automatización y Product Managers para desarrollar soluciones creativas en procesos de localización y productos tecnológicos de la empresa. Debe estar automotivado para aprender y tener interés y experiencia en web scraping."
1,Ciencia_de_datos_España.txt,"Acerca del empleo 🚀 Únete a Inetum como Responsable de Data & AI ¿Eres un líder visionario apasionado por la tecnología y la innovación? Esta es tu oportunidad para dirigir nuestra unidad de negocio de Data & AI, transformando industrias y generando impacto real. 🔍 Lo que harás: Liderarás un equipo multidisciplinar, con un alto volumen de negocio. Diseñarás y ejecutarás estrategias innovadoras de Data & AI para sectores clave como banca, telecomunicaciones, retail, salud, ... Potenciarás relaciones con clientes estratégicos y socios tecnológicos líderes del mercado. Gestionarás el P&L de la unidad, impulsando rentabilidad y excelencia. 🌟 Lo que buscamos: Experiencia liderando unidades de negocio en Data & AI o tecnologías afines en consultoría o integración. Experiencia en áreas como Data Governance, Plataformas y Arquitectura, Visualización e IA. Habilidades excepcionales en desarrollo de negocio y gestión de equipos grandes y diversos. Pasión por la tecnología, el impacto social y la transformación digital. Inglés nivel alto (C1 o superior). 🎯 Lo que ofrecemos: Liderar una unidad clave en un entorno dinámico y en constante evolución. Acceso a proyectos estratégicos con clientes líderes y tecnologías de vanguardia. Retribución competitiva y entorno de trabajo flexible y colaborativo. 📍 Ubicación: Madrid, modalidad de trabajo híbrida. Sé parte de la transformación. ¿List@ para liderar el futuro de Data & AI con nosotros? 🌐"
2,Ciencia_de_datos_España.txt,"Acerca del empleo Estamos ampliando nuestro equipo de ciencia de datos para ayudar a Tendam a resolver desafíos complejos en la industria de la moda retail. Actualmente buscamos un/a Data Scientist para una oportunidad única de influir en la forma en que el mundo experimenta la moda. Qué harás: Descomponer problemas complejos de datos y realizar análisis rigurosos y evaluaciones. Llevar a cabo el preprocesamiento de datos estructurados y no estructurados. Extraer y analizar datos de las bases de datos de la empresa para impulsar la optimización y mejora del desarrollo de productos, técnicas de marketing y estrategias comerciales. Desarrollar modelos de datos y algoritmos personalizados para aplicarlos a conjuntos de datos. Presentar la información mediante técnicas de visualización de datos. Qué buscamos: Formación científica con habilidades computacionales y analíticas. Sólidas habilidades para la resolución de problemas. Conocimientos de programación en Python y SQL. Comprensión y experiencia con sistemas de big data, especialmente Spark (PySpark). Conocimiento de diversas técnicas de aprendizaje automático (clustering, regresión, árboles de decisión, redes neuronales, etc.). Experiencia con herramientas y librerías de visualización de datos. Motivación para aprender y dominar nuevas tecnologías y técnicas. Excelentes habilidades de comunicación oral y escrita. ~1 año de experiencia en la industria. Cualificaciones adicionales: Experiencia trabajando con grandes volúmenes de datos. Experiencia con servicios web en la nube (por ejemplo, AWS). Conocimiento y experiencia con sistemas de control de versiones (por ejemplo, Git). Por qué Tendam: Uno de los mayores repositorios de datos de


Archivo: Ingeniero_de_datos_España


,nombre_archivo,descripcion_oferta
0,Ingeniero_de_datos_España.txt,"Acerca del empleo Los Ingenieros de Software Junior son profesionales técnicamente capacitados con el deseo de diseñar, desarrollar, solucionar problemas y mantener soluciones innovadoras para incidencias técnicas de producción, incluyendo el desarrollo de scripts y aplicaciones para la manipulación de datos y archivos. El candidato ideal es una persona colaborativa que prospera en un entorno de trabajo dinámico, trabajando junto a otros Ingenieros de Software, Ingenieros de Automatización y Product Managers para desarrollar soluciones creativas para procesos de localización y productos tecnológicos de la empresa. Debe tener una actitud proactiva de aprendizaje, con interés y experiencia en web scraping."
1,Ingeniero_de_datos_España.txt,"Acerca del empleo Los Ingenieros de Software Junior son profesionales técnicamente capacitados con el deseo de diseñar, desarrollar, solucionar problemas y mantener soluciones innovadoras para incidencias técnicas de producción, incluyendo el desarrollo de scripts y aplicaciones para la manipulación de datos y archivos. El candidato ideal es una persona colaborativa que prospera en un entorno de trabajo dinámico, trabajando junto a otros Ingenieros de Software, Ingenieros de Automatización y Product Managers para desarrollar soluciones creativas para procesos de localización y productos tecnológicos de la empresa. Debe tener una actitud proactiva de aprendizaje, con interés y experiencia en web scraping."
2,Ingeniero_de_datos_España.txt,"Acerca del empleo Hola, somos LIS Data Solutions, una ingeniería especializada en el análisis de datos, que acompaña y ayuda a las empresas a transformar y desarrollar su negocio con soluciones tecnológicas en proyectos de Industria 4.0, Analítica avanzada de Datos, Big data e Inteligencia Artificial. En LIS tenemos desarrollada una cultura que sitúa a las personas en el centro, y por ello en 2022 hemos sido premiados por la Calidad en el Empleo en Cantabria. Somos una empresa joven, con perspectivas de crecimiento rápido y en la actualidad contamos con oficinas en Vitoria, Santander, Barcelona, Hamburgo y Ulm (Alemania) ¿Qué buscamos? En LIS Data Solutions queremos incorporar DATA ENGINIEERS para ayudar en el desarrollo de nuestras soluciones en nuestras oficinas de Bezana (Cantabria). Descripción de funciones: Trabajar junto al resto de componentes del equipo de proyectos para diseñar y desarrollar nuestras soluciones de datos. Desarrollar plataformas de tratamiento y almacenamiento de datos, con interacción con sistemas de BBDD, asegurando su escalabilidad y rendimiento. Participar en el diseño y desarrollo de scripts de extracción, tratamiento y almacenamiento de datos con Python, Pandas y pySpark sobre Databricks. Trabajar con herramientas de control de versiones como Git. Desarrollar dashboards para la visualización óptima de los datos Participar en la revisión del código y asegurar que los estándares de calidad sean cumplidos. Requisitos deseables: Graduados de ramas técnicas (Ingenieros informáticos, matemáticos, físicos), o en su defecto FPs de ramas DAM con especialización en Big Data. Al menos 1 año de experiencia en desarrollo de ETLs con Python, usando numpy y Pandas. Conocimiento de pySpark y del entorno Databricks. Conocimiento de las soluciones de Azure para el tratamiento de datos en el cloud (Azure Blob Storage, Azure Synapse, etc.) Conocimiento en la utilización de distintas herramientas de visualización como PowerBI, Qlik Sense o Tableau. Manejo con entornos de Bases de Datos, tanto SQL (Postgres, MySQL, SQLServer. Azure SQL) como No SQL (Mongo DB). Familiaridad con Git u otros sistemas de control de versiones. Habilidades de resolución de problemas y depuración de código. Excelentes capacidades de comunicación y capacidad para trabajar en equipo y cliente. Inglés mínimo B2 Si te vienes, te ofrecemos: Plan de carrera personalizado y estable. Contrato indefinido a jornada co


Archivo: Jurista_Málaga


,nombre_archivo,descripcion_oferta
0,Jurista_Málaga.txt,"Acerca del empleo Consejos: Haz un resumen del puesto, explica qué se necesita para triunfar en él y el lugar que ocupa en la empresa. Responsabilidades Buscamos Licenciado/a en económicas / empresariales o similar para trabajar en despacho de asesoría fiscal y contable en régimen de colaboración. Asesoría en fase de crecimiento con necesidad de derivar clientes a colaborador/a. Despacho privado disponible. Contactar para más información y detalles. Requisitos Disponibilidad inmediata. Con experiencia en gestionar cartera de clientes fiscal y contable. Autónomo independiente con experiencia."
1,Jurista_Málaga.txt,"Acerca del empleo En EY tendrás la oportunidad de construir una carrera tan única como tú, en un entorno global de cultura inclusiva y tecnológico. Queremos contar con tu voz, y tu aportación para hacer de EY un sitio mejor. Únete a nosotros y construye una experiencia única y un mundo mejor para todos. La oportunidad EY es líder mundial en prestación de servicios de auditoría, fiscalidad, asesoramiento en transacciones y consultoría, con 4000.000 profesionales en más de 150 países. En España somos más de 6.000 profesionales en 15 oficinas. Nuestro lema es ""Building a better working world ya que nuestro propósito es ayudar a construir un entorno mejor y ser relevantes para nuestros clientes. EY Abogados ofrece un amplio abanico de servicios de asesoramiento fiscal y legal caracterizados por una atención personalizada a nuestros clientes, buscando siempre la solución que se adapte a sus necesidades y estructura, aprovechando las oportunidades y ventajas competitivas a su disposición. Tus funciones principales Formando parte del equipo de Derecho Público y Urbanismo de nuestra oficina de Málaga colaborarás en: Due diligence urbanísticas de todo tipo de activos (suelos y edificación) Tramitación de autorizaciones administrativas urbanísticas (licencias, declaraciones responsables), medioambientales y otras sectoriales para toda clase de activos inmobiliarios. Informes urbanísticos. Seguimiento de iniciativas legislativas en materia urbanística. Secretaría Junta de Compensación y otras entidades urbanísticas. Redacción proyectos de reparcelación y expropiación. Memoria Urbanística de instrumentos de planeamiento. Tramitación de expedientes administrativos. Recursos administrativos y contencioso-administrativo Contratación administrativa Requisitos: Grado en Derecho, Derecho y ADE Valorable máster en Derecho urbanístico. Excelencia técnica Nivel alto de inglés hablado y escrito. Actitudes para el trabajo en equipo. Que Buscamos: Buscamos profesionales con 3 a 5 años de experiencia en el asesoramiento en derecho urbanístico desarrollada en Big Four o despachos de abogados. Que ofrecemos Aprendizaje continuo: Desarrollarás la mentalidad y las habilidades para enfrentarte a nuevos retos. Tú defines el éxito: Te proporcionaremos herramientas y flexibilidad para que puedas llegar a las metas propuestas. Liderazgo transformacional: Te daremos la confianza y formación para que puedas crecer y llegar a ser un buen líder. Cultura inclusiva y diversidad: Cada persona es única y tiene algo que aportar, te daremos voz para ello; toda idea es importante. Si quieres unirte a nosotros, no dudes en aplicar a esta oferta La experiencia en EY es excepcional; constrúyela EY | Building a better working world EY existe para construir un mundo de trabajo mejor, ayudamos a nuestros clientes, personas y sociedad a crear un valor a largo plazo para generar valor en el mercado. Gracias a los datos y la tecnología, tenemos equipos en más de 150 países generando confianza y ayudando a los clientes a crecer, transformarse y operar. Trabajando en Auditoria, Consultoría, Tax, Estrategia y Transacciones, los equipos de EY se hacen las mejores preguntas para encontrar nuevas respuestas a complejos problemas en los que se encuentra nuestro mundo a día de hoy."
2,Jurista_Málaga.txt,"Acerca del emple


Archivo: Traductor_de_inglés_Málaga


,nombre_archivo,descripcion_oferta
0,Traductor_de_inglés_Málaga.txt,"Acerca del empleo Localsoft, S.L. es una empresa de localización de primer nivel que ofrece servicios de traducción, pruebas y audio. Desde nuestra sede en el Málaga TechPark, España, operamos tanto a nivel local como internacional con clientes de diversos sectores, incluyendo marcas líderes en las industrias de videojuegos, tecnología, legal y empresarial. Actualmente estamos contratando traductores experimentados de inglés a alemán (Alemania) para proyectos temporales de traducción in-house de videojuegos AAA, para trabajar en equipo dentro de nuestro Departamento de Localización en nuestra sede en Málaga TechPark, España. Si tienes excelentes habilidades lingüísticas y experiencia en proyectos de localización de videojuegos, ¡nos gustaría conocerte! Como traductor de videojuegos, serás responsable de localizar juegos a tu idioma nativo, asegurando que se cumplan los más altos estándares de calidad en todo momento. Visita: www.localsoftgames.com Requisitos Disponibilidad para participar en un proyecto temporal basado en Málaga Inglés con dominio profesional completo Excelente dominio del idioma nativo (alemán - Alemania) Título universitario en Traducción más 2 años de experiencia a tiempo completo en traducción/revisión en la industria de los videojuegos, o bien 5 años de experiencia a tiempo completo en traducción/revisión en dicha industria (o similar) Capacidad para trabajar en un entorno de oficina multicultural, dinámico y con plazos ajustados Capacidad para seguir instrucciones, listas de control y glosarios Capacidad para localizar un mínimo de 2000 palabras por día cumpliendo con los más altos estándares de calidad y respetando los plazos Buenas habilidades para el trabajo en equipo y relaciones interpersonales, actitud proactiva y positiva (""can do"") Conocimientos de informática: paquete Microsoft Office y herramientas CAT Responsabilidades Seguir los procesos establecidos e instrucciones del proyecto Asumir la responsabilidad total de cada fase del ciclo de vida del proyecto Cumplir exitosamente con tareas y plazos con los más altos estándares de calidad Apoyar y ayudar a los demás miembros del equipo Cumplir con las políticas de confidencialidad y seguridad de Localsoft Beneficios Oportunidad de trabajar con un equipo internacional profesional en un entorno de trabajo dinámico y amigable Oportunidad de colaborar con clientes de renombre (editores/desarrolladores) Oportunidad de trabajar en videojuegos AAA Cafetería y zona de juegos Refrescos y fruta fresca Apoyo para reubicación Información adicional El trabajo se realiza en las oficinas de Localsoft en Málaga, España No es posible el trabajo remoto"
1,Traductor_de_inglés_Málaga.txt,"Acerca del empleo Localsoft, S.L. es una empresa de localización de primer nivel que ofrece servicios de traducción, pruebas y audio. Desde nuestra sede en el Málaga TechPark, España, operamos tanto a nivel local como internacional con clientes de distintos sectores, incluyendo marcas líderes en las industrias de videojuegos, tecnología, legal y empresarial. Actualmente estamos contratando traductores experimentados de inglés a español (ES) para proyectos temporales de traducción in-house de videojuegos AAA, para trabajar en equipo dentro de nuestro Departamento de Localización en nuestra sede en Málaga TechPark, España. Si tienes excelentes habilidades lingüísticas y al menos 3 años de experiencia en localización de videojuegos, ¡nos encantaría conocerte! Como traductor de videojuegos, serás responsable de localizar juegos a tu idioma nativo, asegurando que se cumplan en todo momento los más altos estándares de calidad. Visita: www.localsoftgames.com Requisitos Disponibilidad para participar en un proyecto temporal basado en Málaga Inglés con dominio profesional completo Excelente dominio del idioma nativo (español - España) 3 años de experiencia a tiempo completo en traducción/revisión dentro de la industria de 